In [ ]:
import os
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
import argparse
import matplotlib.pyplot as plt
import torchvision.io as io
from PIL import Image
import torchvision.transforms as transforms
import torch

#### Load Model

In [ ]:
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)  # get test options

In [ ]:
# For CycleGAN
parser.model = "test"
parser.name = "surreal2agora"

# for descriminatro
parser.lambda_identity = 0.5
parser.n_layers_D = 3
parser.pool_size = 50
parser.gan_mode = "lsgan"
parser.lr = 0.0002
parser.beta1 = 0.5
parser.lr_policy = "linear"
parser.lr_decay_iters = 50

parser.gpu_ids = [0]
parser.isTrain = False
parser.checkpoints_dir = "./checkpoints" # the directory where our checkpoints are stored
parser.preprocess = "none"
parser.input_nc = 3
parser.output_nc = 3
parser.ngf = 64
parser.ndf = 64
parser.netD = "basic"
parser.netG = "resnet_9blocks"
parser.norm = "instance"
parser.no_dropout = True
parser.init_type = "normal"
parser.init_gain = 0.02
parser.load_iter = 0
parser.epoch = 113 # the epoch of checkpoint we will be using
parser.verbose = True
parser.num_threads = 0   # test code only supports num_threads = 0
parser.batch_size = 1    # test code only supports batch_size = 1
parser.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
parser.no_flip = True    # no flip; comment this line if results on flipped images are needed.
parser.display_id = -1   # no visdom display; the test code saves the results to a HTML file.

In [ ]:
model = create_model(parser)      # create a model given opt.model and other options
model.setup(parser)               # regular setup: load and print networks; create schedulers
model.eval()

#### Generate Images & Compute Loss

In [ ]:
# Image Generation
# real_surreal: B, 3, 256, 256
fake_agora = model.netG_A(real_surreal)  # B, 3, 256, 256
recon_surreal = model.netG_B(fake_agora)   # B, 3, 256, 256

# Loss
fake_loss = torch.mean(model.netD_B(fake_agora.detach())**2)
real_loss = torch.mean((model.netD_B(real_surreal)-torch.ones_like(model.netD_B(real_surreal)))**2)
recon_loss = torch.mean(torch.abs(recon_surreal - real_surreal))
loss = fake_loss + real_loss + model.lambda_identity*recon_loss
